In [1]:
import requests
import time
from multiprocessing import Pool
import json
import re

In [2]:
def pcode_to_data(pcode):
    if int(pcode) % 1000 == 0:
        print(pcode)
    
    page = 1
    results = []
    
    while True:
        try:
            response = requests.get('http://developers.onemap.sg/commonapi/search?searchVal={0}&returnGeom=Y&getAddrDetails=Y&pageNum={1}'
                                    .format(pcode, page)) \
                               .json()
        except requests.exceptions.ConnectionError as e:
            print('Fetching {} failed. Retrying in 2 sec'.format(pcode))
            
            time.sleep(2)
            continue
            
        results = results + response['results']
    
        if response['totalNumPages'] > page:
            page = page + 1
        else:
            break
            
    return results

In [4]:
pcode_to_data('068809')

[{'SEARCHVAL': 'OUE DOWNTOWN',
  'BLK_NO': '6',
  'ROAD_NAME': 'SHENTON WAY',
  'BUILDING': 'OUE DOWNTOWN',
  'ADDRESS': '6 SHENTON WAY OUE DOWNTOWN SINGAPORE 068809',
  'POSTAL': '68809',
  'X': '29679.40246',
  'Y': '28827.56391',
  'LATITUDE': '1.276980707',
  'LONGITUDE': '103.84840859999998',
  'LONGTITUDE': '103.84840859999998'},
 {'SEARCHVAL': 'OUE DOWNTOWN 2',
  'BLK_NO': '6',
  'ROAD_NAME': 'SHENTON WAY',
  'BUILDING': 'OUE DOWNTOWN 2',
  'ADDRESS': '6 SHENTON WAY OUE DOWNTOWN 2 SINGAPORE 068809',
  'POSTAL': '68809',
  'X': '29742.067089999997',
  'Y': '28931.83176',
  'LATITUDE': '1.277923666',
  'LONGITUDE': '103.8489717',
  'LONGTITUDE': '103.8489717'},
 {'SEARCHVAL': 'OAKWOOD PREMIER OUE SINGAPORE',
  'BLK_NO': '6',
  'ROAD_NAME': 'SHENTON WAY',
  'BUILDING': 'OAKWOOD PREMIER OUE SINGAPORE',
  'ADDRESS': '6 SHENTON WAY OAKWOOD PREMIER OUE SINGAPORE SINGAPORE 068809',
  'POSTAL': '68809',
  'X': '29700.34325',
  'Y': '28865.05368',
  'LATITUDE': '1.2773197509999998',
  'LO

* Pool consuming memory

In [ ]:
if __name__ == '__main__':
    pool = Pool(processes=5)
    
    postal_codes = range(0, 1000000)
    postal_codes = ['{0:06d}'.format(p) for p in postal_codes]

    all_buildings = pool.map(pcode_to_data, postal_codes)
    all_buildings.sort(key=lambda b: (b['POSTAL'], b['SEARCHVAL']))

    jstr = json.dumps([y for x in all_buildings for y in x], indent=2, sort_keys=True)

    with open('buildings.json', 'w') as f:
        f.write(jstr.encode('utf-8'))

* Non Pool 